# <a id='toc1_'></a>[Steam Trike Calculations](#toc0_)
Like many design tasks, sizing the various components of the steam trike is an iterative process. We start with certain assumed parameters and goals, then develop cylinder and accumulator dimensions. These, in turn, affect piping and generator design. At various points in the design, we can compare the results against the initial assumptions and adjust.

In this notebook, we start with one set of assumed parameters as a demonstration, but develop the code in a way that we can evaluate different sets of parameters easily.

**Table of contents**<a id='toc0_'></a>    
- [Steam Trike Calculations](#toc1_)    
  - [Initial Parameters](#toc1_1_)    
    - [Steam Pressure](#toc1_1_1_)    
    - [Vehicle Weight](#toc1_1_2_)    
  - [Required Performance](#toc1_2_)    
    - [Maximum Velocity](#toc1_2_1_)    
    - [Acceleration](#toc1_2_2_)    
      - [Normal Acceleration Data](#toc1_2_2_1_)    
      - [0-60 MPH Time Data](#toc1_2_2_2_)    
      - [Selection](#toc1_2_2_3_)    
  - [Drivetrain Dimensions](#toc1_3_)    
    - [Max RPM / Wheel Diameter](#toc1_3_1_)    
    - [Required Force at Roadway](#toc1_3_2_)    
    - [Required Cylinder Dimensions](#toc1_3_3_)    
  - [Miscellaneous Drivetrain Parameters](#toc1_4_)    
    - [Piston Speed](#toc1_4_1_)    
    - [Crosshead Force](#toc1_4_2_)    
  - [Steam System Sizing](#toc1_5_)    
    - [Maximum Steam Rate](#toc1_5_1_)    
    - [Steam Inlet Pipe Sizing](#toc1_5_2_)    
    - [Steam Exhaust Port/Pipe Sizing](#toc1_5_3_)    
    - [Accumulator Sizing](#toc1_5_4_)    
    - [Steam Generator Sizing](#toc1_5_5_)    
    - [TODO: Generator Coil Dimensions](#toc1_5_6_)    
  - [Evalution of Initial Iteration](#toc1_6_)    
- [Iterations](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
# initial setup
%pip install pint
%pip install pyXSteam
import numpy as np
import pandas as pd
import pint                                 # units library
from pyntXSteam import pyntXSteam           # pint wrapper for steam library pyXSteam
ureg = pint.UnitRegistry()                  # initialize pint units registry for this notebook
ureg.autoconvert_offset_to_baseunit = True  # disable relative temperature units
ureg.default_format = "~0.2fP"              # enable pretty unit outputs
steam = pyntXSteam(ureg)                    # initialize steam library with pint registry
x = {"iteration": "initial"}                # iteration parameters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## <a id='toc1_1_'></a>[Initial Parameters](#toc0_)
### <a id='toc1_1_1_'></a>[Steam Pressure](#toc0_)
The steam pressure for the system directly affects the design of the entire machine. We assume a maximum allowable working pressure (MAWP) of 150 psi; components like tanks, valves, and fittings are readily available with this rating. Max steam pressure in the accumulator is further limited by the fact that ASME Section VIII safety relief valves start venting at 10% below MAWP, or 135 psi. A pressure deadband then defines the operating pressure range for the system and we can find the average pressure.

In [2]:
x['max_pressure'] = 135 * ureg('lbf/in**2')  
x['pressure_deadband'] = 40 * ureg('lbf/in**2')
def avg_pressure(y):
    value = y['max_pressure'] - (y['pressure_deadband'] / 2)

    value = value.to('lbf/in**2')
    y['avg_pressure'] = value
    return value
avg_pressure(x)

115.0 <Unit('force_pound / inch ** 2')>

In [3]:
def min_pressure(y):
    value = y['max_pressure'] - y['pressure_deadband']

    value = value.to('lbf/in**2')
    y['min_pressure'] = value
    return value
min_pressure(x)

95.0 <Unit('force_pound / inch ** 2')>

### <a id='toc1_1_2_'></a>[Vehicle Weight](#toc0_)
As is always the case for iterative design, we have to start with a huge assumption about the system that won't be confirmed or disconfrimed until the other end of the calculation and design process. Based on others' homebuilt vehicle projects, we can assume a target weight, which includes the vehicle, fuel/water, passengers, and any cargo.

In [4]:
x['trike_weight'] = 2000 * ureg('lb')

## <a id='toc1_2_'></a>[Required Performance](#toc0_)
### <a id='toc1_2_1_'></a>[Maximum Velocity](#toc0_)
We select a target maximum velocity based on a reasonable use case, like highway but not freeway travel

In [5]:
x['max_velocity'] = 55 * ureg('mi/hr')

### <a id='toc1_2_2_'></a>[Acceleration](#toc0_)
#### <a id='toc1_2_2_1_'></a>[Normal Acceleration Data](#toc0_)
We want acceleration behavior compatible with traffic, so we can look at published data for conventional road vehicles. The data shown in Figures 1 and 2 are for consumer vehicles accelerating in normal traffic, not the maximum acceleration the vehicles are capable of. Thus, we want to target a value on the high end to reflect extra capacity available for "flooring it".


<div align="center">
<img src="fig5e.PNG" width="400"/><br />
Fig 1. Acceleration vs speed of various gasoline vehicles. Graph, "Scatter plots of acceleration-speed," from P.S.Bokare and A.K.Maurya, Acceleration-Deceleration Behaviour of Various Vehicle Types, page 10, figure 5(e).
</div><br />
<div align="center">
<img src="fig6e.PNG" width="400"/><br />
Fig 2. Idealized acceleration vs speed of gasoline vehicles. Graph, "Idealized plot of acceleration with speed for all vehicle types," from P.S.Bokare and A.K.Maurya, Acceleration-Deceleration Behaviour of Various Vehicle Types, page 11, figure 6(e).
</div><br />


The acceleration curves for these conventional vehicles include gear shifts and will not represent the curve for the steam trike. Instead, we are looking to have a comparable acceleration from a stop.

Candidate `max_accel` = 2 \[m/s^2\] ≅ 6.5 \[ft/s^2\]

#### <a id='toc1_2_2_2_'></a>[0-60 MPH Time Data](#toc0_)
0-60 MPH times for vehicles represent maximum acceleration, which is what we want. However, because car acceleration varies as the car accelerates, you're looking at an average max acceleration instead of the highest instantaneous acceleration. Data from ZeroTo60Times.com.

| Prototype | 0-60 MPH Time (s) | Acceleration (ft/s^2) |
| --- | --- | --- |
| 1998 Chevrolet Metro Hatchback | 15.9 | 5.5 |
| 2014 Nissan Versa Note SV Hatchback | 10.2 | 8.6 |
| 2015 Honda Fit EX | 7.8 | 11.3 |

#### <a id='toc1_2_2_3_'></a>[Selection](#toc0_)
We see that the candidate max_accel from normal vehicle acceleration is lower than the max acceleration of our candidate vehicles, except for the Geo Metro, which matches my personal experiences with the Geo Metro. Given that we are looking for a usable max acceleration target, let's use a value slightly faster than the Geo.

In [6]:
x['max_accel'] = 7.0 * ureg('ft/s**2')

## <a id='toc1_3_'></a>[Drivetrain Dimensions](#toc0_)
### <a id='toc1_3_1_'></a>[Max RPM / Wheel Diameter](#toc0_)
What's the maximum RPM we can expect to be able to achieve with the reciprocating engine? Common practice says 250-500 RPM, but we can check top RPMs of prototypical locomotives and steam cars.

| Prototype | Purpose | Max Speed (mi/hr) | Driver Diameter (in) | rev/min | Piston Stroke (in) | Max Piston Speed (ft/s) |
| --- | --- | --- | --- | --- | --- | --- |
| UP 844 | Passenger | 120 | 80 | 504 | 32 | 11.2 |
| UP 4014 | Freight | 80 | 68 | 395 | 32 | 8.8 |
| Reading 2102 | Freight | 80 | 70 | 384 | 32 | 8.5 |
| PM 1225 | Freight | 70 | 69 | 341 | 34 | 8.0 |
| N&W 611 | Passenger | 110 | 70 | 528 | 32 | 11.7 |
| SP 4449 | Passenger | 110 | 80 | 462 | 32 | 10.3 |
| SF 3751 | Passenger | 100 | 80 | 420 | 30 | 8.8 |
| NYC & HRR 999 | Passenger | 100 | 86.5 | 388 | 24 | 6.5 |
| GTW 5629 | Passenger | 80 | 73 | 368 | 28 | 7.2 |
| Stanley Steam Car | Automobile | 60 | (N/A) | 900 | 5 | 3.1 |
| Doble Steam Car | Automobile | 70 | (N/A) | 900 | 4 | 2.5 |


In [7]:
x['max_rpm'] = 350 * ureg('1/min')           # Conservative bet for larger reciprocating steam engine

With maximum RPM and maximum velocity, we can calculate the wheel diameter.

In [8]:
def wheel_dia(y):
    value = y['max_velocity'] / y['max_rpm'] / np.pi

    value = value.to('in')
    y['wheel_dia'] = value
    return value
wheel_dia(x)

52.821251970155906 <Unit('inch')>

### <a id='toc1_3_2_'></a>[Required Force at Roadway](#toc0_)
Required force on the road to accelerate `trike_weight` by `max_accel`

In [9]:
def road_force(y):
    value = y['trike_weight'] * y['max_accel']

    value = value.to('lbf')
    y['road_force'] = value
    return value
road_force(x)

435.13330240194153 <Unit('force_pound')>

### <a id='toc1_3_3_'></a>[Required Cylinder Dimensions](#toc0_)
The mean tractive force of a two cylinder engine, at 90 percent cutoff, with the cylinders 90 degrees out of phase is given by:

F = K2 * K3 * avg_pressure * cyl_volume / wheel_dia
* K2 = 1.2 - The ratio of mean torque of two identical piston-crank assemblies 90 degrees out of phase to the peak torque of one of the piston-crank assemblies (Jeffrey Hook, Fundamentals of Steam Locomotive Tractive Force, page 7)
* K3 = 0.9 - A fudge factor to account for pressure loss between the boiler and the cylinder (Hook)
* cyl_volume - The swept volume of the cylinder

We will solve for the swept volume of the cylinder,

In [10]:
def cyl_volume(y):
    value = road_force(y) * wheel_dia(y) * 1.2 * 0.9 / y['avg_pressure']

    value = value.to('in**3')
    y['cyl_volume'] = value
    return value
cyl_volume(x)

215.852423228881 <Unit('inch ** 3')>

We set cylinder bore as a parameter and calculate stroke.

In [11]:
x['cyl_bore'] = 5.0 * ureg('in')
def cyl_stroke(y):
    value = cyl_volume(y) / (y['cyl_bore']**2 * np.pi / 4)

    value = value.to('in')
    y['cyl_stroke'] = value
    return value
cyl_stroke(x)

10.993273643276884 <Unit('inch')>

## <a id='toc1_4_'></a>[Miscellaneous Drivetrain Parameters](#toc0_)
### <a id='toc1_4_1_'></a>[Piston Speed](#toc0_)
Max piston speed is important for seal selection.

In [12]:
def max_piston_speed(y):
    value = y['max_rpm'] * cyl_stroke(y) / 2

    value = value.to('ft/s')
    y['max_piston_speed'] = value
    return value
max_piston_speed(x)


2.671976232740909 <Unit('foot / second')>

### <a id='toc1_4_2_'></a>[Crosshead Force](#toc0_)
We can find the maximum force on the crosshead by specifying a connecting rod length. This is important for designing the crosshead bearing. 

In [13]:
x['conn_rod_length'] = 52 * ureg('in')
def max_crosshead_force(y):
    max_piston_force = (y['avg_pressure'] + (y['pressure_deadband'] / 2)) * (y['cyl_bore'] ** 2) * np.pi / 4
    max_angle = np.arctan((y['cyl_stroke'] / 2)/ y['conn_rod_length'])
    max_moment = max_piston_force * y['conn_rod_length'] * np.sin(max_angle)
    value = max_moment / (y['conn_rod_length'] * np.cos(max_angle))

    value = value.to('lbf')
    y['max_crosshead_force'] = value
    return value
max_crosshead_force(x)

280.1930493836436 <Unit('force_pound')>

## <a id='toc1_5_'></a>[Steam System Sizing](#toc0_)
### <a id='toc1_5_1_'></a>[Maximum Steam Rate](#toc0_)
Maximum steam rate occurs at max power output, so we need to define a reasonable target. Let's define max power as the power required to go up a target slope at full speed.

In the vicininity of Austin, Ranch Road 2222 encounters a hill of 10% slope for most of a mile as shown in Figure 3. Not far from there, Figure 4 shows that Spicewood Springs has a 15% slope for ~500 ft. Many vehicles struggle to maintain speed going up that slope. Let's design for the longer slope and allow the energy in the accumulator to handle short bursts like Spicewood Springs.

<div align="center">
<img src="2222slope.PNG" width="400"/><br />
Fig 3. Google Earth Pro elevation profile for RM 2222 between Loop 360 and FM 620.
</div><br />
<div align="center">
<img src="spicewoodslope.PNG" width="400"/><br />
Fig 4. Google Earth Pro elevation profile for Spicewood Springs Road near Loop 360.
</div><br />

We neglect air resistance for this calculation, and assume the throttle is full open but cutoff has been linked up. This means that on each stroke, a certain volume of steam is admitted at the full pressure available to the cylinder, then allowed to expand to an exhaust pressure. This represents the best possible efficiency. We can assume isentropic expansion to find the mass of steam required to produce the energy per stroke required to maintain speed up the target slope.

In [14]:

x['max_road_slope'] = 0.10
def max_steam_mass_rate(y):
    slope_rad = np.arctan(y['max_road_slope'])
    # convert trike_weight to force (assume Earth)
    slope_force = (y['trike_weight'].to('lb').magnitude * ureg('lbf')) * np.sin(slope_rad)
    slope_power = slope_force * y['max_velocity']
    stroke_energy = slope_power / y['max_rpm'] / 4
    # convert avg_pressure into absolute pressure, 
    # taking into account pressure loss between boiler and cylinder per Hook's approximation
    p1 = y['avg_pressure'] * 0.9 + (1 * ureg('atm'))
    p2 = 1 * ureg('atm') + 10 * ureg('lbf/in**2')
    u1 = steam.uV_p(p1)
    # assume isentropic expansion
    s = steam.sV_p(p1)
    u2 = steam.u_ps(p2, s)
    stroke_mass = stroke_energy / (u1 - u2)
    value =  stroke_mass * 4 * y['max_rpm']
    
    value = value.to('lb/min')
    y['max_steam_mass_rate'] = value
    return value
max_steam_mass_rate(x)

11.948473635663618 <Unit('pound / minute')>

In [15]:
def max_steam_liquid_rate(y):
    value = max_steam_mass_rate(y) * steam.v_pt(1 * ureg('atm'), 25 * ureg('degC'))
    
    value = value.to('gal/min')
    y['mass_steam_liquid_rate'] = value
    return value
max_steam_liquid_rate(x)

1.435981880251367 <Unit('gallon / minute')>

### <a id='toc1_5_2_'></a>[Steam Inlet Pipe Sizing](#toc0_)
Standard practice is to size steam lines according to velocity. Spirax Sarco's [piping guide](https://www.spiraxsarco.com/learn-about-steam/steam-distribution/pipes-and-pipe-sizing) suggests a range of 25-40 m/s (82-131 ft/s). We can establish a minimum internal diameter for total flow through a single pipe (for example, for flow out of the accumulator), as well as for double pipes (after flow is split to each cylinder). 

In [16]:
def single_inlet_pipe_dia(y):
    specific_volume = steam.vV_p(y['min_pressure'] + 1 * ureg('atm'))
    area = max_steam_mass_rate(y) * specific_volume / (131 * ureg('ft/s'))
    value = np.sqrt(4 * area / np.pi)

    value = value.to('in')
    y['single_pipe_dia'] = value
    return value
single_inlet_pipe_dia(x)


1.0637991281329275 <Unit('inch')>

In [17]:
def double_inlet_pipe_dia(y):
    specific_volume = steam.vV_p(y['min_pressure'] + 1 * ureg('atm'))
    area = max_steam_mass_rate(y) / 2 * specific_volume / (131 * ureg('ft/s'))
    value = np.sqrt(4 * area / np.pi)

    value = value.to('in')
    y['double_pipe_dia'] = value
    return value
double_inlet_pipe_dia(x)

0.7522195773231302 <Unit('inch')>

### <a id='toc1_5_3_'></a>[Steam Exhaust Port/Pipe Sizing](#toc0_)
At exhaust, the steam can be assumed to be around ~10 psi. Since the cylinder ports must pass both inlet and exhaust steam, they are sized for the larger area required for exhaust. We can also calculate the pipe diameter needed for each cylinder's exhaust line.

In [18]:
def cyl_port_area(y):
    specific_volume = steam.vV_p(10 * ureg('psi') + 1 * ureg('atm'))
    value = max_steam_mass_rate(y) / 2 * specific_volume / (131 * ureg('ft/s'))

    value = value.to('in**2')
    y['cyl_port_area'] = value
    return value
cyl_port_area(x)

1.8053152085046982 <Unit('inch ** 2')>

In [19]:
def double_exhaust_pipe_dia(y):
    specific_volume = steam.vV_p(10 * ureg('psi') + 1 * ureg('atm'))
    area = max_steam_mass_rate(y) / 2 * specific_volume / (131 * ureg('ft/s'))
    value = np.sqrt(4 * area / np.pi)

    value = value.to('in')
    y['double_pipe_dia'] = value
    return value
double_exhaust_pipe_dia(x)

1.5161130281677513 <Unit('inch')>

### <a id='toc1_5_4_'></a>[Accumulator Sizing](#toc0_)
We can use the mass flow at average boiler pressure to determine how long the accumulator will last at max steam rate when discharging from the top to the bottom of the pressure deadband.

In [20]:
x['accum_capacity'] = 15 * ureg('gal')
def accum_steam_mass(y):
    max_p = y['avg_pressure'] + y['pressure_deadband'] / 2
    min_p = y['avg_pressure'] - y['pressure_deadband'] / 2
    # assume accumulator is 90% full of saturated water when charged
    liquid_volume = y['accum_capacity'] * 0.9
    liquid_mass = liquid_volume / steam.vL_p(max_p)
    # using equation 3.22.1 from the Spirax Sarco guide
    delta_h = steam.hL_p(max_p) - steam.hL_p(min_p)
    value = delta_h * liquid_mass / (steam.hV_p(min_p) - steam.hL_p(min_p))

    value = value.to('lb')
    y['accum_steam_mass'] = value
    return value
accum_steam_mass(x)

3.0707300331927545 <Unit('pound')>

In [21]:
def accum_buffer_time(y):
    value = accum_steam_mass(y) / max_steam_mass_rate(y)

    value = value.to('s')
    y['accum_buffer_time'] = value
    return value
accum_buffer_time(x)

15.41986094706166 <Unit('second')>

The Spirax Sarco [accumulator sizing guide](https://www.spiraxsarco.com/learn-about-steam/the-boiler-house/steam-accumulators) includes an empirical relationship between available saturated water surface area, pressure, and the maximum steam release rate in an accumulator without entraining water. We use this to find the required surface area in the accumulator required to support max steam rate.

In [22]:

def accum_surface_area(y):
    area = max_steam_mass_rate(y).to('kg/hr').magnitude / 220 / (y['avg_pressure'].to('bar').magnitude + 1)
    value = (area * ureg('m**2'))

    value = value.to('ft**2')
    y['accum_surface_area'] = value
    return value
accum_surface_area(x)

1.7818675032514566 <Unit('foot ** 2')>

### <a id='toc1_5_5_'></a>[Steam Generator Sizing](#toc0_)
Next, we find the heat input required to boil water that water (at constant pressure, as the feed pump/injector handles that), assuming some heat transfer efficiency for the generator coil.

In [23]:

x['burner_power'] = 700000 * ureg('Btu/hr')
x['gen_coil_efficiency'] = 0.7
def gen_mass_rate(y):
    init_h = steam.h_pt(y['avg_pressure'] + 1 * ureg('atm'), 70 * ureg('degF'))   # liquid phase
    final_h = steam.hV_p(y['avg_pressure'] + 1 * ureg('atm'), 1 * ureg(''))    # dry vapor phase
    heat_transfer_rate = y['burner_power'] * y['gen_coil_efficiency']
    value = heat_transfer_rate / (final_h - init_h)

    value = value.to('lb/min')
    y['gen_mass_rate'] = value
    return value
gen_mass_rate(x)

7.079014881379253 <Unit('pound / minute')>

With the average generator production, we can estimate how long it takes to charge the accumulator from "fully discharged".

In [24]:
def accum_charge_time(y):
    value = accum_steam_mass(y) / gen_mass_rate(y)

    value = value.to('s')
    y['accum_charge_time'] = value
    return value
accum_charge_time(x)


26.026757264800068 <Unit('second')>

### <a id='toc1_5_6_'></a>[TODO: Generator Coil Dimensions](#toc0_)

## <a id='toc1_6_'></a>[Evalution of Initial Iteration](#toc0_)
Now that we have calculated a single design iteration, let's evaluate the results.

In [25]:
df = pd.DataFrame(x, index=[1])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df

,iteration,max_pressure,pressure_deadband,avg_pressure,min_pressure,trike_weight,max_velocity,max_accel,max_rpm,wheel_dia,road_force,cyl_volume,cyl_bore,cyl_stroke,max_piston_speed,conn_rod_length,max_crosshead_force,max_road_slope,max_steam_mass_rate,mass_steam_liquid_rate,single_pipe_dia,double_pipe_dia,cyl_port_area,accum_capacity,accum_steam_mass,accum_buffer_time,accum_surface_area,burner_power,gen_coil_efficiency,gen_mass_rate,accum_charge_time
1,initial,135.00 lbf/in²,40.00 lbf/in²,115.00 lbf/in²,95.00 lbf/in²,2000.00 lb,55.00 mi/h,7.00 ft/s²,350.00 1/min,52.82 in,435.13 lbf,215.85 in³,5.00 in,10.99 in,2.67 ft/s,52.00 in,280.19 lbf,0.1,11.95 lb/min,1.44 gal/min,1.06 in,1.52 in,1.81 in²,15.00 gal,3.07 lb,15.42 s,1.78 ft²,700000.00 Btu/h,0.7,7.08 lb/min,26.03 s


* Wheel diameter - Road legal tires up to 53" are commercially available, but they are heavy, expensive, and wide
* Cylinder dimensions - Performing any machining operations on the 10 inch axis of the cylinder block is going to be difficult on any readily-available machine, but finding and working with stock that supports a 5 inch or larger bore will be difficult too
* Max Steam Heat Rate - 1.5 million BTU/hr is over twice what the largest DC burner available can put out, and that's not even including the inefficiency of heat transfer to the coil

From here, we can tweak assumptions as we like to get a design that looks more reasonable.

# <a id='toc2_'></a>[Iterations](#toc0_)

In [26]:
iters = [
    x,  # 0: initial iteration from text
    {   "iteration": "lighter, higher rpm, smaller bore",
        "trike_weight": 1500 * ureg('lb'),              # 2000 -> 1500 lb
        "max_pressure": 135 * ureg('lbf/in**2'),        
        "pressure_deadband": 40 * ureg('lbf/in**2'),
        "max_velocity": 55 * ureg('mi/hr'),
        "max_rpm": 450 * ureg('1/min'),                 # 350 -> 450 RPM
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 4.5 * ureg('in'),                   # 5 -> 4.5 in
        "conn_rod_length": 36 * ureg('in'),
        "max_road_slope": 0.1,
        "accum_capacity": 15 * ureg('gal'),
        "gen_coil_efficiency": 0.7,
        "burner_power": 700000 * ureg('Btu/hr')
    },
    {   "iteration": "lighter, higher rpm, lower avg pressure, bigger pressure deadband",
        "trike_weight": 1500 * ureg('lb'),              # 2000 -> 1500 lb
        "max_pressure": 135 * ureg('lbf/in**2'),
        "pressure_deadband": 70 * ureg('lbf/in**2'),    # 40 -> 70 psi
        "max_velocity": 55 * ureg('mi/hr'),
        "max_rpm": 450 * ureg('1/min'),                 # 350 -> 450
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 5.0 * ureg('in'),
        "conn_rod_length": 36 * ureg('in'),
        "max_road_slope": 0.1,
        "accum_capacity": 15 * ureg('gal'),
        "gen_coil_efficiency": 0.7,
        "burner_power": 700000 * ureg('Btu/hr')
    },
    {   "iteration": "much smaller, much lighter, much slower",
        "trike_weight": 750 * ureg('lb'),               # 2000 -> 750 lb
        "max_pressure": 135 * ureg('lbf/in**2'),
        "pressure_deadband": 40 * ureg('lbf/in**2'),    # 40 -> 70 psi
        "max_velocity": 35 * ureg('mi/hr'),             # 55 -> 35 mph
        "max_rpm": 450 * ureg('1/min'),                 # 350 -> 450
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 3.25 * ureg('in'),
        "conn_rod_length": 36 * ureg('in'),
        "max_road_slope": 0.05,                         # Dean Keeton St @ UT 
        "accum_capacity": 5 * ureg('gal'),              # 15 -> 5 gal
        "gen_coil_efficiency": 0.7,
        "burner_power": 245000 * ureg('Btu/hr')         # 700k -> 245k Btu/hr
    },
    {   "iteration": "much smaller, much lighter, lower rpm, much slower",
        "trike_weight": 750 * ureg('lb'),               # 2000 -> 750 lb
        "max_pressure": 135 * ureg('lbf/in**2'),
        "pressure_deadband": 40 * ureg('lbf/in**2'),    # 40 -> 70 psi
        "max_velocity": 35 * ureg('mi/hr'),             # 55 -> 35 mph
        "max_rpm": 350 * ureg('1/min'),
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 3.5 * ureg('in'),
        "conn_rod_length": 36 * ureg('in'),
        "max_road_slope": 0.05,                         # Dean Keeton St @ UT 
        "accum_capacity": 5 * ureg('gal'),              # 15 -> 5 gal
        "gen_coil_efficiency": 0.7,
        "burner_power": 245000 * ureg('Btu/hr')         # 700k -> 245k Btu/hr
    },
    {   "iteration": "much smaller, much lighter, lower rpm, much pokier",
        "trike_weight": 750 * ureg('lb'),               # 2000 -> 750 lb
        "max_pressure": 135 * ureg('lbf/in**2'),
        "pressure_deadband": 40 * ureg('lbf/in**2'),    # 40 -> 70 psi
        "max_velocity": 35 * ureg('mi/hr'),             # 55 -> 35 mph
        "max_rpm": 350 * ureg('1/min'),
        "max_accel": 5 * ureg('ft/s**2'),
        "cyl_bore": 3.25 * ureg('in'),
        "conn_rod_length": 36 * ureg('in'),
        "max_road_slope": 0.05,                         # Dean Keeton St @ UT 
        "accum_capacity": 5 * ureg('gal'),              # 15 -> 5 gal
        "gen_coil_efficiency": 0.7,
        "burner_power": 245000 * ureg('Btu/hr')         # 700k -> 245k Btu/hr
    }
]

In [27]:
# run calculations
for each in iters:
    avg_pressure(each)
    min_pressure(each)
    max_piston_speed(each)
    max_crosshead_force(each)
    max_steam_liquid_rate(each)
    single_inlet_pipe_dia(each)
    double_inlet_pipe_dia(each)
    cyl_port_area(each)
    double_exhaust_pipe_dia(each)
    accum_buffer_time(each)
    accum_surface_area(each)
    accum_charge_time(each)
# display
df = pd.DataFrame(iters)
df.transpose()

,0,1,2,3,4,5
iteration,initial,"lighter, higher rpm, smaller bore","lighter, higher rpm, lower avg pressure, bigger pressure deadband","much smaller, much lighter, much slower","much smaller, much lighter, lower rpm, much slower","much smaller, much lighter, lower rpm, much pokier"
max_pressure,135.00 lbf/in²,135.00 lbf/in²,135.00 lbf/in²,135.00 lbf/in²,135.00 lbf/in²,135.00 lbf/in²
pressure_deadband,40.00 lbf/in²,40.00 lbf/in²,70.00 lbf/in²,40.00 lbf/in²,40.00 lbf/in²,40.00 lbf/in²
avg_pressure,115.00 lbf/in²,115.00 lbf/in²,100.00 lbf/in²,115.00 lbf/in²,115.00 lbf/in²,115.00 lbf/in²
min_pressure,95.00 lbf/in²,95.00 lbf/in²,65.00 lbf/in²,95.00 lbf/in²,95.00 lbf/in²,95.00 lbf/in²
trike_weight,2000.00 lb,1500.00 lb,1500.00 lb,750.00 lb,750.00 lb,750.00 lb
max_velocity,55.00 mi/h,55.00 mi/h,55.00 mi/h,35.00 mi/h,35.00 mi/h,35.00 mi/h
max_accel,7.00 ft/s²,7.00 ft/s²,7.00 ft/s²,7.00 ft/s²,7.00 ft/s²,5.00 ft/s²
max_rpm,350.00 1/min,450.00 1/min,450.00 1/min,450.00 1/min,350.00 1/min,350.00 1/min
wheel_dia,52.82 in,41.08 in,41.08 in,26.14 in,33.61 in,33.61 in
